In [ ]:
from pathlib import Path

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from misc import model_config

In [2]:
main_model_config = (
    model_config.query("main")
    .drop(columns="main")
    .rename(columns={k: f"model_{k}" for k in model_config.columns})
)

new_name = {
    "powermoe": "PowerMoE",
    "llamamoe": "LLaMA-MoE-v1",
    "olmoe": "OLMoE",
    "switch": "SwitchTransformers",
    "llamamoe2": "LLaMA-MoE-v2",
    "jetmoe": "JetMoE",
    "openmoe": "OpenMoE",
    "minicpm": "MiniCPM-MoE",
    "qwen": "Qwen1.5-MoE",
    "deepseek2": "DeepSeek-V2-Lite",
    "deepseek": "DeepSeekMoE",
    "xverse": "XVERSE-MoE",
    "qwen3": "Qwen3",
    "yuan": "Yuan2.0",
    "phi": "Phi-3.5-MoE",
    "grin": "GRIN-MoE",
    "mixtral": "Mixtral-8x7B",
    "jamba": "Jamba-Mini",
    "nllb": "NLLB-MoE",
    "qwen2": "Qwen2",
}

model_colors = {
    key: px.colors.qualitative.Dark24[i] for i, key in enumerate(main_model_config.index.values)
}

seg_lens = (4, 16, 64, 256)
seg_len_colors = {key: px.colors.qualitative.Plotly[i] for i, key in enumerate(seg_lens)}
main_model_config

,model_name,model_abbr,model_type,model_num_params,model_num_layers,model_num_experts,model_top_k,model_attn
key,,,,,,,,
powermoe,PowerMoE-3B,PW,causal,3.30,32,40,8,flash_attention_2
llamamoe,LLaMA-MoE-v1-3.5B,LL1,causal,6.74,32,16,4,eager
olmoe,OLMoE-1B-7B-0125,OL,causal,6.92,16,64,8,flash_attention_2
switch,SwitchTransformers-Base-128,ST,seq2seq,7.42,24,128,1,eager
llamamoe2,LLaMA-MoE-v2-3.8B,LL2,causal,8.03,32,8,2,flash_attention_2
jetmoe,JetMoE-8B,JT,causal,8.52,24,8,2,flash_attention_2
openmoe,OpenMoE-8B,OP,causal,11.86,24,32,2,eager
minicpm,MiniCPM-MoE-8x2B,MC,causal,13.87,40,8,2,flash_attention_2
qwen,Qwen1.5-MoE-A2.7B,QW1,causal,14.32,24,60,4,flash_attention_2


In [3]:
def make_abbr(df):
    return (
        f"{df['model_abbr']}{'d' if df['is_decoder'] else 'e'}"
        if df["model_type"] == "seq2seq"
        else df["model_abbr"]
    )

In [ ]:
root_dir = Path("../output/srp_pos_mpq")

dfs = {
    p.stem: pd.merge(pd.read_parquet(p), main_model_config, left_on="model", right_index=True)
    for p in root_dir.glob("*.parquet")
}

for df in dfs.values():
    df["model"] = df["model"].astype(model_config.index.dtype)

dfs["mg"]

,model,is_decoder,seg_len,start_pos,act_r,best_f1,best_m,model_name,model_abbr,model_type,model_num_params,model_num_layers,model_num_experts,model_top_k,model_attn
0,powermoe,True,4,0,0.200,0.616218,1.025195,PowerMoE-3B,PW,causal,3.30,32,40,8,flash_attention_2
1,powermoe,True,4,1,0.200,0.668296,1.089514,PowerMoE-3B,PW,causal,3.30,32,40,8,flash_attention_2
2,powermoe,True,4,2,0.200,0.673125,1.096706,PowerMoE-3B,PW,causal,3.30,32,40,8,flash_attention_2
3,powermoe,True,4,3,0.200,0.674550,1.098278,PowerMoE-3B,PW,causal,3.30,32,40,8,flash_attention_2
4,powermoe,True,4,4,0.200,0.675213,1.100563,PowerMoE-3B,PW,causal,3.30,32,40,8,flash_attention_2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48045,qwen2,True,256,252,0.125,0.275368,2.530671,Qwen2-57B-A14B,QW2,causal,57.41,28,64,8,flash_attention_2
48046,qwen2,True,256,253,0.125,0.275366,2.530599,Qwen2-57B-A14B,QW2,causal,57.41,28,64,8,flash_attention_2
48047,qwen2,True,256,254,0.125,0.275364,2.530506,Qwen2-57B-A14B,QW2,causal,57.41,28,64,8,flash_attention_2
48048,qwen2,True,256,255,0.125,0.275366,2.530530,Qwen2-57B-A14B,QW2,causal,57.41,28,64,8,flash_attention_2


In [5]:
smgdf = (
    dfs["mg"]
    .groupby(["model", "is_decoder", "seg_len"], as_index=False, observed=True)[["best_f1"]]
    .std()
)

smgdf.pivot(index=["model", "is_decoder"], columns="seg_len", values="best_f1")

seg_len                    4         16        64        256
model     is_decoder                                        
powermoe  True        0.002574  0.000756  0.000314  0.000138
llamamoe  True        0.002251  0.001232  0.000644  0.000197
olmoe     True        0.004865  0.006085  0.005223  0.002647
switch    False       0.002649  0.002792  0.002069  0.000584
          True        0.013866  0.008928  0.000835       NaN
llamamoe2 True        0.006526  0.003851  0.001854  0.000809
jetmoe    True        0.001859  0.001641  0.001258  0.000504
openmoe   True        0.007175  0.014517  0.009706  0.004261
minicpm   True        0.002201  0.001795  0.001372  0.000552
qwen      True        0.002048  0.003713  0.002968  0.001334
deepseek2 True        0.006391  0.007014  0.005442  0.002828
deepseek  True        0.005878  0.005510  0.004925  0.002457
xverse    True        0.008094  0.008471  0.006704  0.003433
qwen3     True        0.007445  0.006051  0.004887  0.002271
yuan      True        0.003460  0.001099  0.000573  0.000315
phi       True        0.004435  0.001751  0.000619  0.000274
grin      True        0.004115  0.002510  0.001988  0.001069
mixtral   True        0.000638  0.000621  0.000547  0.000199
jamba     True        0.007086  0.006877  0.005288  0.002911
nllb      False       0.008160  0.004198  0.012322  0.016452
          True        0.003501  0.013795  0.017524  0.015634
qwen2     True        0.004104  0.001208  0.000234  0.000180

In [6]:
sample_seg_len = 16

smddf = (
    dfs["md"]
    .query(f"seg_len == {sample_seg_len}")
    .drop(columns="seg_len")
    .groupby(["model", "is_decoder", "dataset"], as_index=False, observed=True)[["best_f1"]]
    .std()
)

smddf.pivot(index=["model", "is_decoder"], columns="dataset", values="best_f1")

dataset                     c4    cc2306      book  wikipedia     arxiv  \
model     is_decoder                                                      
powermoe  True        0.000962  0.000894  0.000945   0.000931  0.001202   
llamamoe  True        0.000784  0.000811  0.000856   0.000950  0.001065   
olmoe     True        0.001936  0.001909  0.002129   0.002130  0.009202   
switch    False       0.002670  0.002880  0.003012   0.004250  0.003537   
          True        0.011998  0.011939  0.012141   0.007719  0.008127   
llamamoe2 True        0.004573  0.004275  0.004853   0.002754  0.004945   
jetmoe    True        0.000857  0.000813  0.001045   0.001840  0.001525   
openmoe   True        0.015577  0.014170  0.013227   0.011982  0.016763   
minicpm   True        0.000858  0.000973  0.001209   0.001851  0.001937   
qwen      True        0.001428  0.002061  0.002060   0.003159  0.004384   
deepseek2 True        0.002711  0.002686  0.002009   0.004577  0.009264   
deepseek  True        0.001876  0.001802  0.002007   0.003408  0.006698   
xverse    True        0.003828  0.004240  0.003231   0.005539  0.008726   
qwen3     True        0.003320  0.003256  0.004211   0.003171  0.006121   
yuan      True        0.001459  0.001245  0.001619   0.001604  0.001876   
phi       True        0.002551  0.002380  0.003361   0.001780  0.002786   
grin      True        0.002351  0.002288  0.003348   0.001581  0.004440   
mixtral   True        0.000642  0.000472  0.000450   0.000452  0.000870   
jamba     True        0.002605  0.003132  0.003089   0.007885  0.005952   
nllb      False       0.001988  0.002289  0.002366   0.005201  0.006812   
          True        0.014629  0.014273  0.017007   0.018826  0.013515   
qwen2     True        0.000608  0.000673  0.000674   0.001515  0.001570   

dataset               stackexchange    github   lmarena      math      code  \
model     is_decoder                                                          
powermoe  True             0.000969  0.001341  0.001008  0.001214  0.001030   
llamamoe  True             0.001368  0.002167  0.001707  0.001497  0.001142   
olmoe     True             0.007762  0.014043  0.005710  0.008160  0.008573   
switch    False            0.003223  0.003628  0.003465  0.002074  0.001870   
          True             0.008674  0.007738  0.007896  0.007286  0.009359   
llamamoe2 True             0.003722  0.004105  0.003023  0.002880  0.002784   
jetmoe    True             0.001649  0.003165  0.002207  0.001871  0.001622   
openmoe   True             0.014949  0.013020  0.012163  0.019367  0.016962   
minicpm   True             0.001738  0.002963  0.002559  0.002161  0.001984   
qwen      True             0.004701  0.006945  0.004562  0.004049  0.004163   
deepseek2 True             0.008251  0.013087  0.007995  0.008813  0.008803   
deepseek  True             0.006784  0.011656  0.007242  0.007681  0.007598   
xverse    True             0.011651  0.017554  0.010308  0.007987  0.009759   
qwen3     True             0.007841  0.011022  0.005100  0.017965  0.011956   
yuan      True             0.001453  0.001252  0.001656  0.001580  0.001265   
phi       True             0.002077  0.002993  0.001683  0.003107  0.002109   
grin      True             0.002923  0.006316  0.002734  0.008220  0.006215   
mixtral   True             0.000837  0.001509  0.000898  0.001083  0.000972   
jamba     True             0.008270  0.009789  0.008185  0.010971  0.010249   
nllb      False            0.004823  0.006757  0.006590  0.007350  0.005419   
          True             0.012026  0.016150  0.013519  0.012764  0.012411   
qwen2     True             0.001270  0.001887  0.001783  0.001298  0.001198   

dataset                science  
model     is_decoder            
powermoe  True        0.001095  
llamamoe  True        0.001798  
olmoe     True        0.005582  
switch    False       0.002746  
          True        0.009324  
llamamoe2 True        0.005690  
jetmoe    True        0.002372  
openm

In [7]:
sorted_model_keys = (
    pd.merge(
        pd.read_parquet("./output/srp_mpq/mg.parquet"),
        main_model_config,
        left_on="model",
        right_index=True,
    )
    .query("is_decoder and seg_len == 16")
    .sort_values("best_f1", ascending=False)["model"]
)

sorted_model_keys

36     llamamoe2
84          yuan
1       powermoe
80         qwen3
88           phi
13         olmoe
92          grin
96       mixtral
60       minicpm
44        jetmoe
5       llamamoe
76        xverse
100        jamba
68     deepseek2
72      deepseek
112        qwen2
108         nllb
64          qwen
56       openmoe
33        switch
Name: model, dtype: category
Categories (27, object): ['powermoe' < 'llamamoe' < 'llamamoes' < 'olmoe' ... 'mixtral' < 'jamba' < 'nllb' < 'qwen2']

In [8]:
mgdf = (
    dfs["mg"]
    .assign(seg_pos=dfs["mg"]["start_pos"] + dfs["mg"]["seg_len"] // 2)
    .drop(columns="start_pos")
)

mgdf

,model,is_decoder,seg_len,act_r,best_f1,best_m,model_name,model_abbr,model_type,model_num_params,model_num_layers,model_num_experts,model_top_k,model_attn,seg_pos
0,powermoe,True,4,0.200,0.616218,1.025195,PowerMoE-3B,PW,causal,3.30,32,40,8,flash_attention_2,2
1,powermoe,True,4,0.200,0.668296,1.089514,PowerMoE-3B,PW,causal,3.30,32,40,8,flash_attention_2,3
2,powermoe,True,4,0.200,0.673125,1.096706,PowerMoE-3B,PW,causal,3.30,32,40,8,flash_attention_2,4
3,powermoe,True,4,0.200,0.674550,1.098278,PowerMoE-3B,PW,causal,3.30,32,40,8,flash_attention_2,5
4,powermoe,True,4,0.200,0.675213,1.100563,PowerMoE-3B,PW,causal,3.30,32,40,8,flash_attention_2,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48045,qwen2,True,256,0.125,0.275368,2.530671,Qwen2-57B-A14B,QW2,causal,57.41,28,64,8,flash_attention_2,380
48046,qwen2,True,256,0.125,0.275366,2.530599,Qwen2-57B-A14B,QW2,causal,57.41,28,64,8,flash_attention_2,381
48047,qwen2,True,256,0.125,0.275364,2.530506,Qwen2-57B-A14B,QW2,causal,57.41,28,64,8,flash_attention_2,382
48048,qwen2,True,256,0.125,0.275366,2.530530,Qwen2-57B-A14B,QW2,causal,57.41,28,64,8,flash_attention_2,383


In [10]:
num_cols = 10
num_rows = (len(main_model_config) - 1) // num_cols + 1

fig = make_subplots(
    rows=num_rows,
    cols=num_cols,
    shared_xaxes="all",
    shared_yaxes="all",
    horizontal_spacing=0.005,
    vertical_spacing=0.11,
    subplot_titles=sorted_model_keys.map(new_name).values,
)

font_size = [16, 16, 18, 20]
show_legend = True

for i, key in enumerate(sorted_model_keys):
    if (mgdf["model"] == key).sum() == 0:
        continue

    row = i // num_cols + 1
    col = i % num_cols + 1
    num_layers = model_config.loc[key, "num_layers"]

    for seg_len in seg_lens:
        tmpdf = mgdf.query(f"model == '{key}' and seg_len == {seg_len}")
        if len(tmpdf) == 0:
            continue

        line_name = f"SRP(E,{seg_len})"

        if main_model_config.loc[key, "model_type"] == "seq2seq":
            for is_decoder in (False, True):
                subdf = tmpdf.query(f"is_decoder == {is_decoder}")
                if len(subdf) == 0:
                    continue

                fig.add_scatter(
                    x=subdf["seg_pos"],
                    y=subdf["best_f1"],
                    hoverinfo="skip",
                    legendgroup=line_name,
                    line=go.scatter.Line(
                        color=seg_len_colors[seg_len], dash="solid" if is_decoder else "dot"
                    ),
                    mode="lines",
                    name=line_name,
                    opacity=1 if is_decoder else 0.5,
                    showlegend=show_legend,
                    row=row,
                    col=col,
                )
        else:
            fig.add_scatter(
                x=tmpdf["seg_pos"],
                y=tmpdf["best_f1"],
                hoverinfo="skip",
                legendgroup=line_name,
                line=go.scatter.Line(color=seg_len_colors[seg_len]),
                mode="lines",
                name=line_name,
                showlegend=show_legend,
                row=row,
                col=col,
            )

    show_legend = False

    fig.update_xaxes(
        showticklabels=True,
        tickfont=go.layout.xaxis.Tickfont(size=font_size[1]),
        row=row,
        col=col,
    )

    if row == num_rows:
        fig.update_xaxes(
            title=go.layout.xaxis.Title(
                font=go.layout.xaxis.title.Font(size=font_size[2]), standoff=1, text="Position"
            ),
            row=row,
            col=col,
        )

    fig.update_yaxes(showticklabels=col == 1, row=row, col=col)

    if col == 1:
        fig.update_yaxes(
            tickfont=go.layout.yaxis.Tickfont(size=font_size[1]),
            title=go.layout.yaxis.Title(
                font=go.layout.yaxis.title.Font(size=font_size[2]), text="SRP(E,m)"
            ),
            row=row,
            col=col,
        )

fig.update_annotations(font=go.layout.annotation.Font(size=font_size[3]))

fig.update_layout(
    legend=go.layout.Legend(
        font=go.layout.legend.Font(size=font_size[2]), orientation="h", x=0.47, xanchor="center"
    ),
    margin=go.layout.Margin(l=60, r=15, t=30, b=15),
    width=2000,
    height=500,
)

fig.write_image("./plot/msrpp.pdf", width=fig.layout.width, height=fig.layout.height)
fig.show()